### 데이터 분석

In [47]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import  OrdinalEncoder
from sklearn.ensemble import ExtraTreesClassifier

In [48]:
train_data = pd.read_csv('./data/train.csv').drop(columns=['ID'])
test_data = pd.read_csv('./data/test.csv').drop(columns=['ID'])

In [49]:
print(train_data.shape)

(256351, 68)


In [50]:
### 결측치가 많은 컬럼 찾기
missing_data = train_data.isnull().sum() # 컬럼별 결측치 개수 확인
missing_ratio = (missing_data) / len(train_data) * 100 # 결측치 비율 계산
print(missing_ratio[missing_ratio>30])

임신 시도 또는 마지막 임신 경과 연수    96.344855
착상 전 유전 검사 사용 여부         98.939735
PGD 시술 여부                99.149994
PGS 시술 여부                99.247516
난자 해동 경과일                99.439831
배아 해동 경과일                84.252451
dtype: float64


In [51]:
### 범주형 변수 찾기
categorical_columns = train_data.select_dtypes(include=['object']).columns

# 범주형 변수 개수 출력
print(f"범주형 변수 개수: {len(categorical_columns)}")
print(categorical_columns)

# 범주형 변수별 고유값 개수 출력
for col in categorical_columns:
    print(f"{col}: {train_data[col].nunique()}개")


범주형 변수 개수: 20
Index(['시술 시기 코드', '시술 당시 나이', '시술 유형', '특정 시술 유형', '배란 유도 유형', '배아 생성 주요 이유',
       '총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수',
       'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수', '난자 출처',
       '정자 출처', '난자 기증자 나이', '정자 기증자 나이'],
      dtype='object')
시술 시기 코드: 7개
시술 당시 나이: 7개
시술 유형: 2개
특정 시술 유형: 24개
배란 유도 유형: 4개
배아 생성 주요 이유: 13개
총 시술 횟수: 7개
클리닉 내 총 시술 횟수: 7개
IVF 시술 횟수: 7개
DI 시술 횟수: 7개
총 임신 횟수: 7개
IVF 임신 횟수: 7개
DI 임신 횟수: 6개
총 출산 횟수: 7개
IVF 출산 횟수: 6개
DI 출산 횟수: 5개
난자 출처: 3개
정자 출처: 4개
난자 기증자 나이: 5개
정자 기증자 나이: 7개


In [52]:
### 데이터 분포 확인
print(train_data.describe())

       임신 시도 또는 마지막 임신 경과 연수       배란 자극 여부    단일 배아 이식 여부  착상 전 유전 검사 사용 여부  \
count            9370.000000  256351.000000  250060.000000            2718.0   
mean                9.270651       0.771286       0.233476               1.0   
std                 3.550313       0.420005       0.423043               0.0   
min                 0.000000       0.000000       0.000000               1.0   
25%                 7.000000       1.000000       0.000000               1.0   
50%                 9.000000       1.000000       0.000000               1.0   
75%                11.000000       1.000000       0.000000               1.0   
max                20.000000       1.000000       1.000000               1.0   

       착상 전 유전 진단 사용 여부     남성 주 불임 원인     남성 부 불임 원인     여성 주 불임 원인  \
count     250060.000000  256351.000000  256351.000000  256351.000000   
mean           0.012781       0.028516       0.013115       0.030724   
std            0.112328       0.166441       0.113767       0.1

### 결측 데이터 제거

In [53]:
train_data = train_data.drop(columns=['임신 시도 또는 마지막 임신 경과 연수'])
train_data = train_data.drop(columns=['착상 전 유전 검사 사용 여부'])
train_data = train_data.drop(columns=['PGD 시술 여부'])
train_data = train_data.drop(columns=['PGS 시술 여부'])
train_data = train_data.drop(columns=['난자 해동 경과일'])

In [54]:
print(train_data.shape)

(256351, 63)


### 범주 데이터 형태 변환

In [55]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# ✅ (1) Label Encoding (이진형 변수)
le = LabelEncoder()
train_data['배란 유도 유형'] = le.fit_transform(train_data['배란 유도 유형'])

# ✅ (2) One-Hot Encoding (고유값 개수 10개 이하)
train_data = pd.get_dummies(train_data, columns=['난자 출처', '정자 출처', '시술 시기 코드', 
                                                 '총 시술 횟수', '총 임신 횟수', '총 출산 횟수'], drop_first=True)

# ✅ (3) Target Encoding (고유값 개수 10개 이상)
mean_target = train_data.groupby('특정 시술 유형')['임신 성공 여부'].mean()
train_data['특정 시술 유형'] = train_data['특정 시술 유형'].map(mean_target)

# ✅ (4) 숫자로 변환 가능한 컬럼 처리
train_data['난자 기증자 나이'] = pd.to_numeric(train_data['난자 기증자 나이'], errors='coerce')
train_data['정자 기증자 나이'] = pd.to_numeric(train_data['정자 기증자 나이'], errors='coerce')

# ✅ 변환 완료된 데이터 확인
print(train_data.head())

  시술 당시 나이 시술 유형  특정 시술 유형  배란 자극 여부  배란 유도 유형  단일 배아 이식 여부  착상 전 유전 진단 사용 여부  \
0  만18-34세   IVF  0.272825         1         0          0.0               0.0   
1  만45-50세   IVF  0.272825         0         3          0.0               0.0   
2  만18-34세   IVF  0.261457         1         0          0.0               0.0   
3  만35-37세   IVF  0.272825         1         0          0.0               0.0   
4  만18-34세   IVF  0.272825         1         0          0.0               0.0   

   남성 주 불임 원인  남성 부 불임 원인  여성 주 불임 원인  ...  총 임신 횟수_3회  총 임신 횟수_4회  \
0           0           0           0  ...       False       False   
1           0           0           0  ...       False       False   
2           0           0           0  ...       False       False   
3           0           0           0  ...       False       False   
4           0           0           0  ...       False       False   

   총 임신 횟수_5회  총 임신 횟수_6회 이상  총 출산 횟수_1회  총 출산 횟수_2회  총 출산 횟수_3회  총 출산 횟수_4회  \
0       Fals

In [56]:
print(train_data.shape)

(256351, 86)
